# Pommerman V4 Training in smaller central region

This notebook demonstrates how to train Pommerman agents. Please let us know at support@pommerman.com if you run into any issues.

In [1]:
import os
import sys
import numpy as np
import time

import pommerman
from pommerman.agents import SimpleAgent, RandomAgent, PlayerAgent, BaseAgent
from pommerman.configs import ffa_v4_fast_env
from pommerman.envs.v4 import Pomme as Pomme_v4
from pommerman.characters import Bomber
from pommerman import utility
from pommerman import agents
from pommerman import envs
from pommerman import constants
from pommerman import characters

# print all env configs
print(pommerman.REGISTRY)

['PommeFFACompetition-v0', 'PommeFFACompetitionFast-v0', 'PommeFFAFast-v0', 'PommeFFA-v1', 'PommeFFAFast-v3', 'PommeFFAFast-v4', 'OneVsOne-v0', 'PommeRadioCompetition-v2', 'PommeRadio-v2', 'PommeTeamCompetition-v0', 'PommeTeamCompetitionFast-v0', 'PommeTeamCompetition-v1', 'PommeTeam-v0', 'PommeTeamFast-v0', 'PommeTeamSimple-v0']


# Train with stable baseline

In [2]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2

/Users/kman/opt/anaconda3/envs/pommerman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kman/opt/anaconda3/envs/pommerman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kman/opt/anaconda3/envs/pommerman/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kman/opt/anaconda3/en

## Inherit pommerman env and make it compatible with stable-baseline

In [3]:
class CustomPomme(Pomme_v4):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.obs_raw = None # store the raw version of observation
        self.training_idx = 1 # idx of the agent being trained
    
    # function to flatten pommerman observation
    def _transform_obs(self, obs_raw):
        obs_training = obs_raw[self.training_idx] # default the first agent to be trained

        # construct flattened observation
        obs = [
            *np.array(obs_training["board"]).reshape(-1),
            *np.array(obs_training["bomb_blast_strength"]).reshape(-1),
            *np.array(obs_training["bomb_life"]).reshape(-1),
            *np.array(obs_training["position"]).reshape(-1),
            obs_training["ammo"],
            obs_training["blast_strength"],
            obs_training["can_kick"],
            obs_training["teammate"].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 1 v 1
            obs_training["enemies"][0].value,
            obs_training["enemies"][0].value,
            
            # uncommon if training 2 v 2
#             obs_training["enemies"][1].value,
#             obs_training["enemies"][2].value,
        ]
        return obs
    
    def get_obs_raw(self):
        return self.obs_raw

    def step(self, action_training):
        action_nontraining = self.act(self.obs_raw)
        actions = [*action_nontraining, action_training]
        obs_raw, reward, done, info = super().step(actions)
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw), reward[self.training_idx], done, info
    
    def reset(self):
        obs_raw = super().reset()
        self.obs_raw = obs_raw
        return self._transform_obs(obs_raw)
    
    def render(self,
               mode=None,
               close=False,
               record_pngs_dir=None,
               record_json_dir=None,
               do_sleep=True):
        super().render(mode=mode,
                       close=close,
                       record_pngs_dir=record_pngs_dir,
                       record_json_dir=record_json_dir,
                       do_sleep=do_sleep)

In [4]:
# def team_v3_fast_env():
#     """Start up a FFA config with the default settings."""
#     env = CustomPomme
#     game_type = constants.GameType.Team
#     env_entry_point = 'CustomPomme'
#     env_id = 'PommeTeamFast-v3'
#     env_kwargs = {
#         'game_type': game_type,
#         'board_size': 8,
#         'num_rigid': 0,
#         'num_wood': 0,
#         'num_items': 0,
#         'max_steps': constants.MAX_STEPS,
#         'render_fps': 1000,
#         'env': env_entry_point,
#     }
#     agent = characters.Bomber
#     return locals()

# def one_vs_one_v3_env():
#     """Start up a FFA config with the default settings."""
#     env = CustomPomme
#     game_type = constants.GameType.OneVsOne
#     env_entry_point = 'CustomPomme'
#     env_id = 'PommeOneVsOneFast-v3'
#     env_kwargs = {
#         'game_type': game_type,
#         'board_size': 8,
#         'num_rigid': 0,
#         'num_wood': 0,
#         'num_items': 0,
#         'max_steps': constants.MAX_STEPS,
#         'render_fps': 1000,
#         'env': env_entry_point,
#     }
#     agent = characters.Bomber
#     return locals()

def one_vs_one_v4_env():
    """Start up a FFA config with the default settings."""
    env = CustomPomme
    game_type = constants.GameType.OneVsOne
    env_entry_point = 'CustomPomme'
    env_id = 'PommeOneVsOneFast-v4'
    env_kwargs = {
        'game_type': game_type,
        'board_size': 11,
        'free_board_size': 4,
        'num_rigid': 0,
        'num_wood': 0,
        'num_items': 0,
        'max_steps': constants.MAX_STEPS,
        'render_fps': 1000,
        'env': env_entry_point,
    }
    agent = characters.Bomber
    return locals()

In [5]:
# Instantiate the environment

config = one_vs_one_v4_env()
env_pom = CustomPomme(**config["env_kwargs"])

# config agents
agents = []

# Add simple agents
for agent_id in range(1):
    agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))
    
# add player agent(to train)
agents.append(PlayerAgent(config["agent"](1, config["game_type"])))

env_pom.set_agents(agents)
env_pom.set_training_agent(agents[1].agent_id)
env_pom.set_init_game_state(None)

# Seed and reset the environment
env_pom.seed(0)

[0]

In [6]:
# log function during training, implement if needed
def log(local_var, global_var):
    pass
#     display(local_var)
#     display(global_var)

In [14]:
import time
n_cpu = 2
env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])

model = PPO2(MlpPolicy, env, verbose=1, 
             n_steps = 3000, # batch_size = n_step * num_env
             ent_coef = 0.001, # entropy coefficient
             tensorboard_log="./ppo_pommerman_tensorboard/")
startTime = time.time()
model = model.learn(total_timesteps=4000000, # num_update = total_timesteps // batch_size
                    callback = log)
endTime = time.time()
elapsedTime = endTime - startTime
print(elapsedTime)
# model.save("ppo2_pommerman_20000_2")
# model = model.learn(total_timesteps=5000000, # num_update = total_timesteps // batch_size
#                     callback = log)
model.save("ppo2_pommerman_v4_4")

--------------------------------------
| approxkl           | 0.00021511447 |
| clipfrac           | 0.0           |
| explained_variance | -0.484        |
| fps                | 404           |
| n_updates          | 1             |
| policy_entropy     | 1.7915729     |
| policy_loss        | -0.0009018528 |
| serial_timesteps   | 3000          |
| time_elapsed       | 2.15e-06      |
| total_timesteps    | 6000          |
| value_loss         | 0.11618769    |
--------------------------------------
--------------------------------------
| approxkl           | 0.0005002811  |
| clipfrac           | 0.0           |
| explained_variance | -0.227        |
| fps                | 406           |
| n_updates          | 2             |
| policy_entropy     | 1.7897758     |
| policy_loss        | -0.0012685414 |
| serial_timesteps   | 6000          |
| time_elapsed       | 14.9          |
| total_timesteps    | 12000         |
| value_loss         | 0.081075415   |
-------------------------

---------------------------------------
| approxkl           | 0.00077191653  |
| clipfrac           | 0.0            |
| explained_variance | 0.00917        |
| fps                | 434            |
| n_updates          | 18             |
| policy_entropy     | 1.4725542      |
| policy_loss        | -0.00049163174 |
| serial_timesteps   | 54000          |
| time_elapsed       | 244            |
| total_timesteps    | 108000         |
| value_loss         | 0.079081364    |
---------------------------------------
--------------------------------------
| approxkl           | 0.0028776783  |
| clipfrac           | 0.038458332   |
| explained_variance | -0.000838     |
| fps                | 435           |
| n_updates          | 19            |
| policy_entropy     | 1.4518452     |
| policy_loss        | -0.0023887653 |
| serial_timesteps   | 57000         |
| time_elapsed       | 258           |
| total_timesteps    | 114000        |
| value_loss         | 0.0720797     |
------------

--------------------------------------
| approxkl           | 0.00048358174 |
| clipfrac           | 0.0           |
| explained_variance | 0.0301        |
| fps                | 437           |
| n_updates          | 34            |
| policy_entropy     | 1.334669      |
| policy_loss        | -0.0005592419 |
| serial_timesteps   | 102000        |
| time_elapsed       | 466           |
| total_timesteps    | 204000        |
| value_loss         | 0.07963874    |
--------------------------------------
--------------------------------------
| approxkl           | 0.0039963606  |
| clipfrac           | 0.05245833    |
| explained_variance | 0.0179        |
| fps                | 436           |
| n_updates          | 35            |
| policy_entropy     | 1.3178447     |
| policy_loss        | -0.0028403876 |
| serial_timesteps   | 105000        |
| time_elapsed       | 479           |
| total_timesteps    | 210000        |
| value_loss         | 0.074807614   |
-------------------------

--------------------------------------
| approxkl           | 0.0010948445  |
| clipfrac           | 0.00041666668 |
| explained_variance | 0.025         |
| fps                | 438           |
| n_updates          | 50            |
| policy_entropy     | 1.3423265     |
| policy_loss        | -0.0009754554 |
| serial_timesteps   | 150000        |
| time_elapsed       | 686           |
| total_timesteps    | 300000        |
| value_loss         | 0.07658155    |
--------------------------------------
--------------------------------------
| approxkl           | 0.0015178178  |
| clipfrac           | 0.030083332   |
| explained_variance | 0.0514        |
| fps                | 437           |
| n_updates          | 51            |
| policy_entropy     | 1.3533132     |
| policy_loss        | -0.0012429515 |
| serial_timesteps   | 153000        |
| time_elapsed       | 699           |
| total_timesteps    | 306000        |
| value_loss         | 0.08401557    |
-------------------------

--------------------------------------
| approxkl           | 0.004042419   |
| clipfrac           | 0.032791667   |
| explained_variance | 0.0577        |
| fps                | 436           |
| n_updates          | 67            |
| policy_entropy     | 1.2495171     |
| policy_loss        | -0.0017528718 |
| serial_timesteps   | 201000        |
| time_elapsed       | 919           |
| total_timesteps    | 402000        |
| value_loss         | 0.079835884   |
--------------------------------------
-------------------------------------
| approxkl           | 0.0042106602 |
| clipfrac           | 0.029041668  |
| explained_variance | 0.0338       |
| fps                | 436          |
| n_updates          | 68           |
| policy_entropy     | 1.2715764    |
| policy_loss        | -0.002246154 |
| serial_timesteps   | 204000       |
| time_elapsed       | 933          |
| total_timesteps    | 408000       |
| value_loss         | 0.08106926   |
-------------------------------------

...

---------------------------------------
| approxkl           | 0.00086643157  |
| clipfrac           | 0.007          |
| explained_variance | 0.396          |
| fps                | 434            |
| n_updates          | 589            |
| policy_entropy     | 0.028241735    |
| policy_loss        | -0.00037153473 |
| serial_timesteps   | 1767000        |
| time_elapsed       | 8.27e+03       |
| total_timesteps    | 3534000        |
| value_loss         | 0.08806822     |
---------------------------------------
--------------------------------------
| approxkl           | 0.00050912343 |
| clipfrac           | 0.0056666667  |
| explained_variance | 0.384         |
| fps                | 437           |
| n_updates          | 590           |
| policy_entropy     | 0.031784058   |
| policy_loss        | -0.000206747  |
| serial_timesteps   | 1770000       |
| time_elapsed       | 8.29e+03      |
| total_timesteps    | 3540000       |
| value_loss         | 0.0882097     |
------------

---------------------------------------
| approxkl           | 0.00035585312  |
| clipfrac           | 0.005208333    |
| explained_variance | 0.397          |
| fps                | 437            |
| n_updates          | 606            |
| policy_entropy     | 0.035527684    |
| policy_loss        | -0.00013612914 |
| serial_timesteps   | 1818000        |
| time_elapsed       | 8.51e+03       |
| total_timesteps    | 3636000        |
| value_loss         | 0.088676944    |
---------------------------------------
---------------------------------------
| approxkl           | 0.00033711246  |
| clipfrac           | 0.004833333    |
| explained_variance | 0.407          |
| fps                | 434            |
| n_updates          | 607            |
| policy_entropy     | 0.035927445    |
| policy_loss        | -3.7906153e-05 |
| serial_timesteps   | 1821000        |
| time_elapsed       | 8.52e+03       |
| total_timesteps    | 3642000        |
| value_loss         | 0.08870351     |


---------------------------------------
| approxkl           | 0.001161335    |
| clipfrac           | 0.00825        |
| explained_variance | 0.363          |
| fps                | 433            |
| n_updates          | 622            |
| policy_entropy     | 0.02691583     |
| policy_loss        | -0.00042047168 |
| serial_timesteps   | 1866000        |
| time_elapsed       | 8.73e+03       |
| total_timesteps    | 3732000        |
| value_loss         | 0.08923784     |
---------------------------------------
--------------------------------------
| approxkl           | 0.00063529855 |
| clipfrac           | 0.0057499995  |
| explained_variance | 0.384         |
| fps                | 437           |
| n_updates          | 623           |
| policy_entropy     | 0.027515084   |
| policy_loss        | -0.0005176809 |
| serial_timesteps   | 1869000       |
| time_elapsed       | 8.74e+03      |
| total_timesteps    | 3738000       |
| value_loss         | 0.09033298    |
------------

---------------------------------------
| approxkl           | 0.00032326416  |
| clipfrac           | 0.0036250004   |
| explained_variance | 0.38           |
| fps                | 434            |
| n_updates          | 638            |
| policy_entropy     | 0.027280817    |
| policy_loss        | -0.00024420783 |
| serial_timesteps   | 1914000        |
| time_elapsed       | 8.95e+03       |
| total_timesteps    | 3828000        |
| value_loss         | 0.08519528     |
---------------------------------------
---------------------------------------
| approxkl           | 0.00038898407  |
| clipfrac           | 0.0037083337   |
| explained_variance | 0.384          |
| fps                | 437            |
| n_updates          | 639            |
| policy_entropy     | 0.02650403     |
| policy_loss        | -0.00010391166 |
| serial_timesteps   | 1917000        |
| time_elapsed       | 8.96e+03       |
| total_timesteps    | 3834000        |
| value_loss         | 0.08523044     |


--------------------------------------
| approxkl           | 0.00085632433 |
| clipfrac           | 0.009249999   |
| explained_variance | 0.399         |
| fps                | 406           |
| n_updates          | 654           |
| policy_entropy     | 0.03352984    |
| policy_loss        | 0.00032894363 |
| serial_timesteps   | 1962000       |
| time_elapsed       | 9.17e+03      |
| total_timesteps    | 3924000       |
| value_loss         | 0.0913123     |
--------------------------------------
--------------------------------------
| approxkl           | 0.00056624704 |
| clipfrac           | 0.0047083334  |
| explained_variance | 0.401         |
| fps                | 420           |
| n_updates          | 655           |
| policy_entropy     | 0.03332037    |
| policy_loss        | 2.5919213e-05 |
| serial_timesteps   | 1965000       |
| time_elapsed       | 9.19e+03      |
| total_timesteps    | 3930000       |
| value_loss         | 0.086989366   |
-------------------------

In [16]:
# del model # remove to demonstrate saving and loading
model = PPO2.load("ppo2_pommerman_v4_4")

n_cpu = 1
env = DummyVecEnv([lambda: env_pom for i in range(n_cpu)])
model.envs = env

# test the learned model
num_win = 0
num_tie = 0
num_lose = 0
total = 5 # number of playouts
for i_episode in range(total):
    obs = env.reset()
    done = False
    info = None
    while not done:
        env.render()
        action_training, _states = model.predict(obs)
#         print(action_training)
        obs, rewards, dones, infos = env.step(action_training)
#         print(infos)
        done = dones[0]
        info = infos[0]
        time.sleep(0.1)
    print('Episode {} finished'.format(i_episode))
    if(info["result"].value == 0):
        if(1 in info["winners"]):
            num_win+=1
        else:
            num_lose+=1
    elif(info["result"].value == 2):
        num_tie+=1
#     print(info)
env.close()
print("Win ", num_win, "/", total, " games")
print("Tie ", num_tie, "/", total, " games")
print("Lose ", num_lose, "/", total, " games")


Loading a model without an environment, this model cannot be trained until it has a valid environment.
Episode 0 finished
Episode 1 finished
Episode 2 finished
Episode 3 finished
Episode 4 finished
Win  0 / 5  games
Tie  1 / 5  games
Lose  4 / 5  games


# baseline example code

In [ ]:
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2

# multiprocess environment
n_cpu = 1
env = DummyVecEnv([lambda: gym.make('CartPole-v1') for i in range(n_cpu)])

model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo2_cartpole")

del model # remove to demonstrate saving and loading

model = PPO2.load("ppo2_cartpole")

# Enjoy trained agent
obs = env.reset()



In [ ]:
print(obs)
print(env.buf_obs[None].shape)
print(env.observation_space)

action, _states = model.predict(obs)
obs, rewards, dones, info = env.step(action)

print(obs)
print(rewards)
print(dones)
print(info)

# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()